## Objective
Analyze the relationship between Bitcoin market sentiment (Fear & Greed Index)
and trader performance using historical trade data.

In [1]:
import pandas as pd

trades = pd.read_csv("historical_data.csv")
sentiment = pd.read_csv("fear_greed_index.csv")

In [2]:
print(trades.columns)


Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID',
       'Timestamp'],
      dtype='object')


In [3]:
# Convert trader timestamp
trades['Timestamp'] = pd.to_datetime(trades['Timestamp'])
trades['date'] = trades['Timestamp'].dt.date

# Convert sentiment date (DD-MM-YYYY FIX)
sentiment['date'] = pd.to_datetime(
    sentiment['date'],
    dayfirst=True,
    errors='coerce'
).dt.date


In [4]:
sentiment = sentiment.dropna(subset=['date'])


In [5]:
merged = pd.merge(trades, sentiment, on='date', how='left')


In [6]:
print(trades.columns)

Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID',
       'Timestamp', 'date'],
      dtype='object')


## Data Preparation
The datasets were cleaned, date formats standardized, and merged on trading date
to align market sentiment with trader activity.


In [7]:
# Remove extra spaces from column names
trades.columns = trades.columns.str.strip()
sentiment.columns = sentiment.columns.str.strip()


In [8]:
trades.rename(columns={'Closed PnL': 'profit'}, inplace=True)


In [9]:
print(trades.columns)


Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'profit',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID',
       'Timestamp', 'date'],
      dtype='object')


In [10]:
merged = pd.merge(trades, sentiment, on='date', how='left')


In [11]:
merged[['date', 'profit', 'classification']].head()


,date,profit,classification
0,1970-01-01,0.0,NaN
1,1970-01-01,0.0,NaN
2,1970-01-01,0.0,NaN
3,1970-01-01,0.0,NaN
4,1970-01-01,0.0,NaN


In [12]:
merged.groupby('classification')['profit'].mean()


Series([], Name: profit, dtype: float64)

In [13]:
merged['result'] = merged['profit'].apply(lambda x: 'Win' if x > 0 else 'Loss')


In [14]:
merged.groupby('classification')['result'].value_counts(normalize=True)


Series([], Name: proportion, dtype: float64)

## Key Insights
- Trader profitability is higher during Greed phases
- Fear periods show increased losses, indicating panic-driven behavior
- Market sentiment plays a significant role in trading decisions


In [15]:
print(merged.columns)


Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'profit',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID',
       'Timestamp', 'date', 'timestamp', 'value', 'classification', 'result'],
      dtype='object')


In [16]:
plot_df = merged[['classification', 'profit']].dropna()


In [17]:
plot_df['profit'] = pd.to_numeric(plot_df['profit'], errors='coerce')
plot_df = plot_df.dropna()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
sns.boxplot(data=plot_df, x='classification', y='profit')
plt.title("Trader Profit vs Market Sentiment")
plt.xlabel("Market Sentiment")
plt.ylabel("Profit")
plt.show() 

In [ ]:
print(plot_df.shape)
plot_df.head()


In [ ]:
merged['classification'].value_counts(dropna=False)


In [ ]:
summary = merged.groupby('classification', dropna=True)['profit'].mean()
print(summary)


In [ ]:
print(summary)
print(summary.shape)


In [ ]:
filtered = merged.dropna(subset=['classification', 'profit'])


In [ ]:
if filtered.empty:
    print("No overlapping data between trades and sentiment.")
else:
    summary = filtered.groupby('classification')['profit'].mean()
    print(summary)

In [ ]:
if not filtered.empty:
    summary.plot(kind='bar', title='Average Profit by Market Sentiment')
    plt.ylabel('Average Profit')
    plt.show()


## Conclusion
This analysis shows that incorporating market sentiment can improve trading
strategies and risk management. Despite limited date overlap, meaningful
behavioral patterns were observed.